# Sorting & Ordering in Polars

### Loading Libraries

In [1]:
# ZipFiles
import zipfile

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import polars as pl
import polars.selectors as cs

# Data Visualization
import hvplot
import seaborn as sns
import altair as alt
import matplotlib.pyplot as plt

# XGBoost
import xgboost as xgb

# Scikit-Learn
from sklearn import decomposition

# Date & Time
from datetime import datetime
from datetime import timedelta

In [2]:
hvplot.extension('matplotlib')

## Loading Data

In [3]:
# Path
path = '/Users/isisromero/desktop/polars/datasets/vehicles.csv'

In [4]:
raw = pl.read_csv(path, null_values=['NA'])

In [5]:
def tweak_auto(df):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'createdOn']
    return (df
            .select(pl.col(cols))
            .with_columns(pl.col('year').cast(pl.Int16),
                          pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
                          pl.col(['displ', 'barrels08']).cast(pl.Float32),
                          pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
                          pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
                          is_automatic=pl.col('trany')                    
                          .str.contains('Automatic')
                          .fill_null('Automatic'),
                          num_gears=pl.col('trany')
                          .str.extract(r'(\d+)')
                          .cast(pl.UInt8)
                          .fill_null(6))
           )

In [6]:
autos = tweak_auto(raw)

In [7]:
print(autos)

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  